In [3]:
from pathlib import Path
from typing import NamedTuple
import math
import random

import pandas as pd
import s2sphere
import osmium
import tqdm

import pyarrow.parquet

from reservoir_sampling import weighted

PLANET_EXTRACT = Path.home() / "datasets" / "osm" / "planet-240513.osm.pbf"
CENTRAL_AMERICA_EXTRACT = Path.home() / "datasets" / "osm" / "central-america-latest.osm.pbf"
ANTARCTICA_EXTRACT = Path.home() / "datasets" / "osm" / "antarctica-latest.osm.pbf"

GMAPS_API_KEY = (Path.home() / "api_keys" / "google_maps.txt").read_text().strip()

def sample_point_from_node_list(node_list):
    # Sample a point uniformly from a list of nodes
    # This is a simple way to sample points from a way
    # The way is assumed to be a list of OSM nodes
    # Each node is a tuple of (lat, lng)
    # The function returns a tuple of (lat, lng)
    node = random.choice(list(node_list))

    # TODO: interpolate between nodes
    return (node.lat, node.lon)

class OsmWeightedSamplingHandler(osmium.SimpleHandler):
    def __init__(self, k, progresshook=None):
        super().__init__()
        self.sampler = weighted.EfraimidisSpirakisSampler(k)
        self.progresshook = progresshook

    def way(self, w):
        self.progresshook(1)

        if not 'highway' in w.tags:
            return

        try:
            length = osmium.geom.haversine_distance(w.nodes)
            sampled_point = sample_point_from_node_list(w.nodes)
        except osmium.InvalidLocationError:
            # Possibly a way where nodes are not in the extract
            return

        self.sampler.observe(sampled_point, length)

class OsmNodeSamplingHandler(osmium.SimpleHandler):
    def __init__(self, k, progresshook=None):
        super().__init__()
        self.sampler = weighted.EfraimidisSpirakisSampler(k)
        self.progresshook = progresshook
        self.count = 0

    def node(self, n):
        self.count += 1
        if self.count % 2048 == 0:
            self.progresshook(2048)

        self.sampler.observe((n.location.lat, n.location.lon), 1)

def sample_osm_length_weighted(k):
    # Sample k random points from ways in the OSM extract
    # The weight of each sample is proportional to the length of the way
    with tqdm.tqdm() as pb:
        handler = OsmWeightedSamplingHandler(k, pb.update)
        #handler.apply_file(PLANET_EXTRACT, locations=True, idx='dense_file_array,planet.nodecache')
        handler.apply_file(CENTRAL_AMERICA_EXTRACT, locations=True, idx='sparse_file_array,central_america.nodecache')
        return handler.sampler.get_samples()

def sample_osm_nodes(k):
    with tqdm.tqdm() as pb:
        handler = OsmNodeSamplingHandler(k, pb.update)
        #handler.apply_file(ANTARCTICA_EXTRACT, locations=False)
        handler.apply_file(PLANET_EXTRACT, locations=False)
        return handler.sampler.get_samples()

def sample_from_way_parquet(parquet_path, k):
    way_reconstructor = ParquetWayReconstructor()
    sampler = weighted.EfraimidisSpirakisSampler(k)

    with pyarrow.parquet.ParquetFile(parquet_path) as pf:
        for batch in pf.iter_batches():
            ways = way_reconstructor.process_batch(batch)
            for way in ways:
                sampled_point = sample_point_from_node_list(way.nodes)
                sampler.observe(sampled_point, way.length)

class SampledSV(NamedTuple):
    lat: float
    lng: float
    density: float

def sv_metadata(lat, lng, radius=50):
    # Call the Street View API to get the nearest panorama
    result = requests.get(
        "https://maps.googleapis.com/maps/api/streetview/metadata"
        f"location={lat},{lng}"
        f"&radius={radius}"
        f"&key={GMAPS_API_KEY}"
    ).json()
    return result

def nearest_sv_with_density(lat, lng):
    result = sv_metadata(lat, lng, radius=1000)
    if result["status"] == "ZERO_RESULTS":
        return None

    # Estimate the density of the matched point, using a few more queries
    for i in range(5):
        direction = random.random() * 2 * math.pi

    return SampledSV()

In [6]:
#sampled_locations = sample_osm_length_weighted(300_000)
sampled_locations = sample_osm_nodes(500_000)
sampled_locations

9172049920it [9:01:16, 282422.81it/s]


[(11.9252812, -15.5335309),
 (46.4992939, 31.9023279),
 (52.4027797, 6.028233),
 (34.6587833, 133.6614407),
 (3.5385687, 12.8351911),
 (65.7357852, 11.5491233),
 (-28.309369, 152.141715),
 (52.2068281, 10.911425),
 (52.442645, -127.1574086),
 (-22.6959173, 17.1707213),
 (27.9192268, -101.2143534),
 (9.5222001, 0.7434133),
 (-19.0945685, -64.2820356),
 (22.3638895, 114.113936),
 (-8.3009888, 114.9472341),
 (-19.8769131, -49.7506795),
 (56.0361774, 12.5867504),
 (45.3258333, 28.7760875),
 (40.8603812, 14.5280661),
 (25.1573419, 55.3246869),
 (41.1043383, -76.1031736),
 (-9.4484241, 34.6673089),
 (40.7507433, 35.599007),
 (-13.9532474, 28.7959626),
 (43.7661446, 7.1864086),
 (36.64004, -81.84612),
 (53.6657379, 78.6193526),
 (47.2802417, 9.9484547),
 (34.5504189, -117.2696929),
 (52.5752053, 22.2184254),
 (23.53152, -164.26097),
 (-20.8911767, 55.4217132),
 (2.3158217, 31.1093801),
 (12.087085, 24.9001493),
 (46.1584596, 15.6500871),
 (53.0967932, 9.9345166),
 (39.9402213, 21.2324867),
 (

In [7]:
df = pd.DataFrame(sampled_locations, columns=["lat", "lng"])
df.to_pickle("sampled_locations.pkl")

In [4]:
df = pd.read_pickle("sampled_locations.pkl")
df

,lat,lng
0,11.925281,-15.533531
1,46.499294,31.902328
2,52.402780,6.028233
3,34.658783,133.661441
4,3.538569,12.835191
...,...,...
499995,50.044020,14.589878
499996,47.100218,1.014429
499997,35.680162,50.967205
499998,42.561284,12.194631


In [11]:
df.sample(frac=1).head(1000).to_pickle(Path.home() / "datasets" / "img2loc" / "outputs" / "world_tiny" / "s1_raw.pkl")

In [8]:
# Plot points on map using Folium
import folium
import itertools

map_ca = folium.Map(location=[37.4301922691736, -122.16943741588071], zoom_start=10)
for point in itertools.islice(df.itertuples(), 0, 5000):
    folium.CircleMarker((point.lat, point.lng), radius=2, color="red").add_to(map_ca)
map_ca